<a href="https://colab.research.google.com/github/abhilash1910/AI-Geometric-Learning/blob/master/Chapter_2_Understanding_the_data/Riemannian_LineSearch_Armijo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Riemmannian Line Search

<img src="https://i.imgur.com/5O09OAq.png">

The Line Search is an optimzation algorithm which tries to minimize an objective function $f(x+\alpha p)$ with respect to $f(x)$ by optimizing the step size. For optimizations in the line search , there are 2 methods:

- Backtracking method starts with a large estimate of $\alpha$  and iteratively shrinks it.
- Adaptive method applies a variational $\alpha$ by Armijo condition which tries to converge faster to the minima

The armijo can be stated as :
Based on a selected control parameter $c \in (0,1)$, the Armijo–Goldstein condition tests whether a step-wise movement from a current position $ x$  to a modified position $x+\alpha p$ achieves an adequately corresponding decrease in the objective function. The condition is fulfilled, if 
$$f(x+\alpha p) \leq f(x) +\alpha c m$$

In this case, we take as inputs:

The Riemannian manifold $M$ with a retraction policy $R$ the projector operator $\pi_{x_{k}}: R^{n}= \tau_{x_{k}}M$ , a real valued differentiable curve(potential function) $f$ with initial iterate $x_{0} \in M$ , the Armijo line search scalar $c$ 

Operation:

while $x_{k}$ does not sufficiently minimize $f$ we have to:
- Take the euclidean gradient to the Riemann direction 
- $\eta_{k} = - grad(f(x_{k})) = \pi_{x_{k}}(- \nabla f(x_{k}))$
- We select $x_{k+1}$ as in Armijo step size $(\tau_{k})$:
   $$f(x_{k})- f(x_{k+1}) \geq c(f(x_{k})- f(R_{x}(\tau_{k} \eta_{k})))$$
- Increment $k= k+1$

In this case, we will be implementing the Armijo condition on Line Search with pytorch for a simpler spherical manifold.






In [ ]:
import torch
import numpy as np

class LineSearchBackTracking:
    """
    Back-tracking line-search based on linesearch
    """
    def __init__(self, contraction_factor=.5, optimism=2,
                 suff_decr=1e-4, maxiter=25, initial_stepsize=1):
        self.contraction_factor = contraction_factor
        self.optimism = optimism
        self.suff_decr = suff_decr
        self.maxiter = maxiter
        self.initial_stepsize = initial_stepsize

        self._oldf0 = None
    def retr(self, X, U):
        Y = X + U
        return Y/torch.norm(Y)

    def objective(self,points,y):
        accumulator = 0
        def dist_(U, V):
          # Make sure inner product is between -1 and 1
          U=torch.tensor(U)
          V=torch.tensor(V)
          inner_pdt=float(torch.dot(U, V))
          inner = max(min(inner_pdt, 1), -1)
          return np.arccos(inner)
        
        for i in range(len(points)):
            accumulator += dist_(y, points[i]) ** 2
        return accumulator / 2

    def search(self, points, x, d, f0, df0):
        """
        Function to perform backtracking line-search.
        Arguments:
            - objective
                objective function to optimise
            - manifold
                manifold to optimise over
            - x
                starting point on the manifold
            - d
                tangent vector at x (descent direction)
            - df0
                directional derivative at x along d
        """
        # Compute the norm of the search direction
        norm_d = torch.norm(x, d)

        if self._oldf0 is not None:
            # Pick initial step size based on where we were last time.
            alpha = 2 * (f0 - self._oldf0) / df0
            # Look a little further
            alpha *= self.optimism
        else:
            alpha = self.initial_stepsize / norm_d
        alpha = float(alpha)

        # Make the chosen step and compute the cost there.
        newx = self.retr(x, alpha * d)
        newf = self.objective(points,newx)
        step_count = 1

        # Backtrack while the Armijo criterion is not satisfied
        while (newf > f0 + self.suff_decr * alpha * df0 and step_count <= self.maxiter):
            # Reduce the step size
            alpha = self.contraction_factor * alpha

            # and look closer down the line
            newx = self.retr(x, alpha * d)
            print("next iterate suggested by the line-search: ",newx)
            newf = self.objective(points,newx)
            step_count = step_count + 1

        # If we got here without obtaining a decrease, we reject the step.
        if newf > f0:
            alpha = 0
            newx = x

        stepsize = alpha * norm_d

        self._oldf0 = f0
        print("Converging at step: ", newx)
        return stepsize, newx

if __name__=='__main__':
  print("Implementation of Riemannian Line Search Optimization by Backtracking on sphere manifold")
  ls=LineSearchBackTracking()
  print("Input coordinates")
  x=torch.tensor([-0.65726771, -0.02678122,  0.7531812])
  print("Parameters for direction of gradient and tangent")
  f0=1
  d=-0.62339641
  df0=0
  points=[[-0.58831187, -0.02677797,  0.80819062],
  [-0.55208236, -0.02669815,  0.83336203],
  [-0.51477841, -0.02656636,  0.85691156],
  [-0.47647259, -0.02638288,  0.87879338],
  [-0.43723948, -0.02614804,  0.89896492],]
  ls.search(points, x, d, f0, df0)





Implementation of Riemannian Line Search Optimization by Backtracking on sphere manifold
Input coordinates
Parameters for direction of gradient and tangent
next iterate suggested by the line-search:  tensor([-0.6003, -0.5787, -0.5520])
next iterate suggested by the line-search:  tensor([-0.6225, -0.5794, -0.5260])
next iterate suggested by the line-search:  tensor([-0.6647, -0.5788, -0.4725])
next iterate suggested by the line-search:  tensor([-0.7379, -0.5698, -0.3618])
next iterate suggested by the line-search:  tensor([-0.8362, -0.5283, -0.1474])
next iterate suggested by the line-search:  tensor([-0.8926, -0.4176,  0.1700])
next iterate suggested by the line-search:  tensor([-0.8507, -0.2684,  0.4520])
Converging at step:  tensor([-0.8507, -0.2684,  0.4520])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [ ]:
class LineSearchAdaptive:
    '''
    Adaptive line-search
    '''

    def __init__(self, contraction_factor=.5, suff_decr=.5, maxiter=10,
                 initial_stepsize=1):
        self._contraction_factor = contraction_factor
        self._suff_decr = suff_decr
        self._maxiter = maxiter
        self._initial_stepsize = initial_stepsize
        self._oldalpha = None

    def retr(self, X, U):
        Y = X + U
        return Y/torch.norm(Y)

    def objective(self,points,y):
        accumulator = 0
        def dist_(U, V):
          # Make sure inner product is between -1 and 1
          U=torch.tensor(U)
          V=torch.tensor(V)
          inner_pdt=float(torch.dot(U, V))
          inner = max(min(inner_pdt, 1), -1)
          return np.arccos(inner)

    def search(self,points, x, d, f0, df0):
        norm_d = man.norm(x, d)

        if self._oldalpha is not None:
            alpha = self._oldalpha
        else:
            alpha = self._initial_stepsize / norm_d
        alpha = float(alpha)

        newx = self.retr(x, alpha * d)
        newf = self.objective(points,newx)
        cost_evaluations = 1

        while (newf > f0 + self._suff_decr * alpha * df0 and
               cost_evaluations <= self._maxiter):
            # Reduce the step size.
            alpha *= self._contraction_factor

            # Look closer down the line.
            newx = self.retr(x, alpha * d)
            newf = self.objective(points,newx)

            cost_evaluations += 1

        if newf > f0:
            alpha = 0
            newx = x

        stepsize = alpha * norm_d

        # Store a suggestion for what the next initial step size trial should
        # be. On average we intend to do only one extra cost evaluation. Notice
        # how the suggestion is not about stepsize but about alpha. This is the
        # reason why this line search is not invariant under rescaling of the
        # search direction d.

        # If things go reasonably well, try to keep pace.
        if cost_evaluations == 2:
            self._oldalpha = alpha
        # If things went very well or we backtracked a lot (meaning the step
        # size is probably quite small), speed up.
        else:
            self._oldalpha = 2 * alpha

        return stepsize, newx

if __name__=='__main__':
  print("Implementation of Adaptive Riemannian Line Search Optimization  on sphere manifold")
  ls=LineSearchBackTracking()
  print("Input coordinates")
  x=torch.tensor([-0.65726771, -0.02678122,  0.7531812])
  print("Parameters for direction of gradient and tangent")
  f0=1
  d=-0.62339641
  df0=0
  points=[[-0.58831187, -0.02677797,  0.80819062],
  [-0.55208236, -0.02669815,  0.83336203],
  [-0.51477841, -0.02656636,  0.85691156],
  [-0.47647259, -0.02638288,  0.87879338],
  [-0.43723948, -0.02614804,  0.89896492],]
  ls.search(points, x, d, f0, df0)



Implementation of Adaptive Riemannian Line Search Optimization  on sphere manifold
Input coordinates
Parameters for direction of gradient and tangent
next iterate suggested by the line-search:  tensor([-0.6003, -0.5787, -0.5520])
next iterate suggested by the line-search:  tensor([-0.6225, -0.5794, -0.5260])
next iterate suggested by the line-search:  tensor([-0.6647, -0.5788, -0.4725])
next iterate suggested by the line-search:  tensor([-0.7379, -0.5698, -0.3618])
next iterate suggested by the line-search:  tensor([-0.8362, -0.5283, -0.1474])
next iterate suggested by the line-search:  tensor([-0.8926, -0.4176,  0.1700])
next iterate suggested by the line-search:  tensor([-0.8507, -0.2684,  0.4520])
Converging at step:  tensor([-0.8507, -0.2684,  0.4520])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
